## Домашнее задание #3: Базы данных и Flask

av Марат Богаутдинов, 232

#### 1.i &emsp; Дизайн таблиц

В целом дизайн таблиц видно в UML-модели, но давайте все равно здесь отдельно пропишу, кто за что отвечает и почему именно такая конфигурация таблиц предлагается:
- сущность **TAG:&emsp;tagID (PK), name**&emsp;$-$ в целом можно было бы спокойно обойтись и без ID, но вдруг когда-нибудь сайт полностью обновит себя, перезапустится и заодно зазахочет обновить все теги, а старые тем не менее сохранить захочет: тогда для одного тега **India** потребуется два айдишника $-$ для старого и нового тега;
- сущность **AUTHOR:&emsp;authorID (PK), name**
- сущность **ARTICLE:&emsp;articleID (PK), title, author (FK), pubDate, link, text**:
- отношение **TAG_ARTICLE:&emsp;tagID (PK, FK), articleID (PK, FK)**
- отношение **AUTHOR_ARTICLE:&emsp;authorID (PK, FK), articleID (PK, FK)**


<img src="https://github.com/mbogautdinov/FCL_projects/blob/main/db_uml.png?raw=1">

#### 1.ii &emsp; Таблицы и связи через Flask-SQLAlchemy

Описание через Flask-SQLAlchemy находится в отдельном файле `models.py`.

Воспользовался инструментом `sqlacodegen`, который автоматически генерирует нужную схему через команду `sqlacodegen sqlite:///archdaily.db > models.py`, запущенную из терминала. Решение нашел на [реддите](https://www.reddit.com/r/learnpython/comments/1avh2i4/generate_python_models_for_sqlalchemy_2x_from/).  

Сами таблицы я создал внутри DB Browser, не через питоновскую библиотеку `sqlite3`.

#### 1.iii &emsp; Добавление данных

Сначала нужно в принципе добавить данные в базу данных. Для это придется немного их обработать:
- склеить единую дату в правильном формате (`YYYY-MM-DD`): пока что есть 3 отдельные колонки $-$ день, месяц и год :)))))
- вытащить все теги ко всем статьям, чтобы каждому дать свой айдишник: пока что у каждой статьи есть свой **список** тегов
- добавить к `href` начало так, чтобы получилась полноценная ссылка 

А пока посмотрим просто на то, как выглядит наш sample:

In [5]:
import pandas as pd

df = pd.read_csv('archdaily_sample.csv')
df.sample(2)

,title,href,full_text,tags,pub_day,pub_month,pub_year,author
670,Finnish Museum of History and Future Selects S...,/1017813/finnish-museum-of-history-and-future-...,Finnish office Sigge Architects has been annou...,"['News', 'Architecture News', 'Sigge Architect...",18,6,2024,Maria-Cristina Florian
926,Designing the Future of Energy: An Exhibition ...,/1015514/designing-the-future-of-energy-an-exh...,The Vitra Design Museum presents 'Transform! D...,"['News', 'Architecture News', 'Vitra Design Mu...",10,4,2024,Maria-Cristina Florian


И начнем его немного ворошить. Во-первых, сделаем адекватную дату. Вообще-то говоря, дата вида `YYYY-MM-DD`::text по своей сути неадекватна $-$ ну как-то грустно в `SQL` не иметь формата данных `date` или `datetime`. По всей видимости, на то она и этот вариант SQLite и light, чтобы особо не утруждать себя поддержкой разнообразных типов данных.  

Ну и ладно. Напишем маленькую функцию, которая создаст новую колонку `pubDate`, а три остальные с информацией о дне, месяце и годе удалим:

In [6]:
df['pubDate'] = df.apply(lambda row: f'{row['pub_year']}-{str(row['pub_month']).zfill(2)}-{str(row['pub_day']).zfill(2)}', axis=1)
df.sample(2)

,title,href,full_text,tags,pub_day,pub_month,pub_year,author,pubDate
1515,Foster + Partners Designs Model Village for Sa...,/1009980/foster-plus-partners-designs-model-vi...,Foster + Partners has been selected to design ...,"['News', 'Architecture News', 'Expo 2025 Osaka...",21,11,2023,Nour Fakharany,2023-11-21
255,Mesmerizing Zigzags: Exploring the Herringbone...,/1001298/mesmerizing-zigzags-exploring-the-her...,Created by a series of angled parallel lines t...,"['News', 'Articles', 'Herringbone', 'ceramics'...",8,10,2024,Camila Prieto,2024-10-08


In [7]:
df = df.drop(labels=['pub_year', 'pub_month', 'pub_day'], axis=1)
# создадим колонку с ID
df['articleID'] = [i for i in range(len(df))]
df.sample(2)

,title,href,full_text,tags,author,pubDate,articleID
132,18th Istanbul Biennial Appoints Christine Tohm...,/1023115/18th-istanbul-biennial-appoints-chris...,The Istanbul Foundation for Culture and Arts (...,"['News', 'Architecture News', 'Istanbul', 'Ist...",Nour Fakharany,2024-11-04,132
1691,The Supporters' Newsletter - Issue #5,/1009689/the-supporters-newsletter-issue-number-5,"At ArchDaily, we face the constant challenge o...","['News', 'Articles']",ArchDaily Team,2023-10-16,1691


Отлично, теперь достанем все теги и запишем их в свой датафрейм:

In [8]:
# сначала приведем в нормальные списки наши теги
df.tags = df.tags.str.strip("[]").str.replace("'", "").str.split(", ")

list_tags = []
for row in range(len(df)):
    list_tags += df.tags[row]

df_tags = pd.DataFrame({'name': list_tags})
df_tags.shape

(28460, 1)

Удалим дупликаты:

In [9]:
df_tags = df_tags.drop_duplicates().reset_index(drop=True)
# создадим колонку с ID
df_tags['tagID'] = [i for i in range(len(df_tags))]
df_tags.shape, df_tags.sample(2)

((4894, 2),
                     name  tagID
 1461      Gottfried Böhm   1461
 909   Warren and Mahoney    909)

Великолепно! теперь то же самое повторим с авторами:

In [10]:
list_authors = []
for row in range(len(df)):
    list_authors += [df.author[row]]

df_authors = pd.DataFrame({'name': list_authors}).drop_duplicates().reset_index(drop=True)
# создадим колонку с ID
df_authors['authorID'] = [i for i in range(len(df_authors))]
df_authors.shape, df_authors.sample(2)

((132, 2),
                name  authorID
 97    Jaume Barnada        97
 24  Romullo Baratto        24)

И последнее: поменяем названия некоторых колонок и добавим к `href` начало `https://www.archdaily.com`:

In [11]:
df = df.rename(columns={'href': 'link', 'full_text': 'text'})
df['link'] = 'https://www.archdaily.com' + df['link']
df.head(3)

,title,link,text,tags,author,pubDate,articleID
0,Evolution of Data Center Design: Modular Const...,https://www.archdaily.com/1024222/evolution-of...,As the AI fervor continues to reshape how peop...,"[News, Articles, Smart Cities, Architecture, D...",Ankitha Gattupalli,2024-12-02,0
1,"Concept, Storytelling, and Visualization: The ...",https://www.archdaily.com/1023896/concept-stor...,"In architecture, the way we communicate our id...","[News, Articles, Architectural Visualization, ...",Enrique Tovar,2024-12-02,1
2,The 25 Tallest Buildings in the World,https://www.archdaily.com/779178/these-are-the...,Humanity has become obsessed with breaking its...,"[News, Articles, Shanghai, Dubai, Changsha, Ch...",Nicolás Valencia,2024-12-02,2


Все, теперь можно приступать к наполнению баз данных данными:

In [12]:
import sqlite3

con = sqlite3.connect('archdaily.db')  # подключение
cur = con.cursor()  # курсор

Теперь попробуем добавить в базу данных данные: сначала из датафреймов `df_tags`, `df_authors` и `df`.

In [13]:
df_tags.to_sql('TAG', con, if_exists='replace', index=False)

4894

In [14]:
df_authors.to_sql('AUTHOR', con, if_exists='replace', index=False)

132

In [15]:
df[['title', 'link', 'text', 'pubDate', 'articleID']].to_sql('ARTICLE', con, if_exists='replace', index=False)

1774

Теперь немного сложнее: заполним таблицы-отношения `AUTHOR_ARTICLE` и `TAG_ARTICLE`

In [16]:
df_auth_art = pd.DataFrame()

for author, articleID in zip(df['author'], df['articleID']):
    # найдем совпадение автора
    author_match = df_authors[df_authors['name'] == author]
    if not author_match.empty:
        # и тут новая строка с конкретным authorID и articleID
        new_row = pd.DataFrame({'authorID': [author_match['authorID'].iloc[0]], 'articleID': [articleID]})
        df_auth_art = pd.concat([df_auth_art, new_row], ignore_index=True)

df_auth_art.sample(4)

,authorID,articleID
1436,116,1436
1238,105,1238
144,9,144
1656,6,1656


In [17]:
df_tag_art = pd.DataFrame()

for tags, articleID in zip(df['tags'], df['articleID']):
    for tag in tags:
        # найдем совпадение тега
        tag_match = df_tags[df_tags['name'] == tag]
        if not tag_match.empty:
            # и тут новая строка с конкретным tagID и articleID
            new_row = pd.DataFrame({'tagID': [tag_match['tagID'].iloc[0]], 'articleID': [articleID]})
            df_tag_art = pd.concat([df_tag_art, new_row], ignore_index=True)

df_tag_art.sample(4)

,tagID,articleID
2143,0,145
21579,103,1316
12578,436,785
20940,1232,1276


Осталось только подгрузить их в базу данных:

In [18]:
df_tag_art.to_sql('TAG_ARTICLE', con, if_exists='replace', index=False)

28460

In [19]:
df_auth_art.to_sql('AUTHOR_ARTICLE', con, if_exists='replace', index=False)

1774

In [20]:
con.commit()

In [21]:
con.close()

#### 1.iii &emsp; Настоящее добавление данных, а не вот эта обманка, которая была выше

Все остальное $-$ в самих файлах!